In [1]:
import h5py
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler


from tools import define_cnn, extract_features, build_model, predict_with_model, set_seed, build_model_and_predict, extract_features_2
import warnings
warnings.filterwarnings('ignore')

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Normal CNN 10-dim vector

In [2]:
# load clean gdp data, keep only year, region and real_gdp columns
ukraine = pd.read_csv("data/tabular_data_ukraine.csv")
log_bin_columns = ["allangle_snow_free_hq" + "_log_" + str(i) for i in range(1, 11)]

# delete Kyiv and Kyiv_Oblast
# ukraine = ukraine[ukraine["region"] != "Kyiv"]
# ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast"]
ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]

# get the data for 2021, 2022 and before 2022
ukraine_2022 = ukraine[ukraine["year"] == 2022]
ukraine = ukraine[ukraine["year"] < 2022]
ukraine_2021 = ukraine[ukraine["year"] == 2021]
ukraine_2021.reset_index(drop=True, inplace=True)
ukraine_2022.reset_index(drop=True, inplace=True)
ukraine.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(ukraine), 765, 1076, 1))
y = np.zeros(len(ukraine))

# load the images
for i in range(len(ukraine)):

    # get year, region, and gdp
    year = ukraine["year"][i]
    region = ukraine["region"][i]
    gdp_value = ukraine["real_gdp"][i]

    # load the image
    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

    # add the values
    y[i] = gdp_value
    X[i, :, :, 0] = allangle_snow_free

# normalise the images and gdp data
maximum_x = X.max()
X = X / maximum_x

maximum_y = y.max()
y = y / maximum_y

# df to store the results
cnn_results = pd.DataFrame(columns = ["mae", "year"])

# define the grid of parameters: network architecture and number of features
param_grid_cnn = {
    'n_features': [4],
    'n_conv': [2],
    'n_dense': [4],
}

param_grid_rf = {
        'n_estimators': [50, 100, 200, 300],
        'max_depth': [5, 10, 15, 20],  # Maximum depth of the tree
        'min_samples_split': [2, 4, 6, 8],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [2, 4, 6, 8],  # Minimum number of samples required to be at a leaf node
        'random_state': [0],  # Ensures reproducibility
}

In [11]:
set_seed(1)
model_cnn = define_cnn(n_features = 4, n_conv = 2, n_dense = 4)
model_cnn.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
model_cnn.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)
cnn_results = pd.DataFrame(columns = ["mae", "year"])

for test_year in range(2013, 2022):
    # get indices for observations in the test and train sets
    test_indices = np.where(ukraine["year"] == test_year)[0]
    train_indices = np.where(ukraine["year"] != test_year)[0]

    # get the train and test sets
    X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]
    ukraine_stage_2, ukraine_test = extract_features_2(model_cnn, ukraine, test_year, X_train, X_test, 4)

    # train first stage, neural network
    for n_features in param_grid_cnn['n_features']:
        for n_conv in param_grid_cnn['n_conv']:
            for n_dense in param_grid_cnn['n_dense']:
                # set_seed(1)
                # model_cnn = define_cnn(n_features = 4, n_conv = n_conv, n_dense = n_dense)
                # model_cnn.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
                # model_cnn.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

                # # extract features
                # ukraine_stage_2, ukraine_test = extract_features_2(model_cnn, ukraine, test_year, X_train, X_test, n_features)

                # train second stage, rf
                train_data = ukraine_stage_2
                test_data = ukraine_test
                selected_columns =  ["feature_" + str(i) for i in range(1, n_features+1)] + log_bin_columns + ['allangle_snow_free_hq_sum']

                # calculate differences
                ukraine = pd.read_csv("data/tabular_data_ukraine.csv")
                ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]
                ukraine_sum = ukraine[['year', 'region', 'allangle_snow_free_hq_sum'] + log_bin_columns]
                ukraine = ukraine[ukraine["year"] < 2022]

                full_data = pd.concat([train_data, test_data])
                full_data.sort_values(by=['region', 'year'], inplace=True)
                full_data = pd.merge(full_data, ukraine_sum, on=['year', 'region'])
                full_data_diff = full_data.groupby('region').diff()
                full_data_diff['region'] = full_data['region']
                full_data_diff['year'] = full_data['year']
                full_data_diff.reset_index(drop=True, inplace=True)
                full_data_diff.dropna(inplace=True)

                train_data_diff = full_data_diff[full_data_diff["year"] != test_year]
                test_data_diff = full_data_diff[full_data_diff["year"] == test_year]
                test_data_diff.reset_index(drop=True, inplace=True)
                train_data_diff.reset_index(drop=True, inplace=True)

                # build the model
                # mae, _, _ = build_model(train_data_diff, test_data_diff, selected_columns, "random_forest", param_grid_rf, 
                #                                                                             log_transform = False, scale = False)

                train_data_nn = train_data_diff.copy()
                pred_data_nn = test_data_diff.copy()

                train_data_nn = pd.get_dummies(train_data_nn, columns=["region"])
                pred_data_nn = pd.get_dummies(pred_data_nn, columns=["region"])

                X_train_nn = train_data_nn.drop(columns=["year", "real_gdp"])
                X_train_nn = X_train_nn.drop(columns=log_bin_columns)
                y_train_nn = train_data_nn["real_gdp"]

                X_test_nn = pred_data_nn.drop(columns=["year", "real_gdp"])
                X_test_nn = X_test_nn.drop(columns=log_bin_columns)
                y_test_nn = pred_data_nn["real_gdp"]

                X_train_nn = np.array(X_train_nn, dtype=np.float32)
                X_test_nn = np.array(X_test_nn, dtype=np.float32)
                y_train_nn = np.array(y_train_nn, dtype=np.float32)
                y_test_nn = np.array(y_test_nn, dtype=np.float32)


                # scale the data
                scaler = StandardScaler()
                X_train_nn = scaler.fit_transform(X_train_nn)
                X_test_nn = scaler.transform(X_test_nn)

                # fit the model on the X_train and y_train
                model = Sequential()
                # model.add(Dense(256, activation="relu", input_dim=X_train_nn.shape[1]))  # First layer
                # model.add(Dense(128, activation="relu"))  # First layer
                # model.add(Dense(64, activation="relu"))  # First layer
                model.add(Dense(32, activation="relu", input_dim=X_train_nn.shape[1]))  # Fourth layer
                model.add(Dense(16, activation="relu"))  # New additional layer
                model.add(Dense(8, activation="relu"))  # New additional layer
                model.add(Dense(1)) 

                model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
                model.fit(X_train_nn, y_train_nn, epochs=500, batch_size=64, validation_split=0.2)

                y_pred = model.predict(X_test_nn).flatten()
                mae = np.mean(np.abs(y_pred - y_test_nn))

                # add the results to the dataframe
                new_row = pd.DataFrame([{
                    "mae": mae,
                    "year": test_year
                }])
                
                cnn_results = pd.concat([cnn_results, new_row], ignore_index=True)
                print("Results for year = ", test_year)

Epoch 1/50
4/4 [==============================] - 5s 873ms/step - loss: 0.0278 - mae: 0.1324 - val_loss: 0.1532 - val_mae: 0.2186
Epoch 2/50
4/4 [==============================] - 2s 495ms/step - loss: 0.0199 - mae: 0.1047 - val_loss: 0.1487 - val_mae: 0.2189
Epoch 3/50
4/4 [==============================] - 2s 457ms/step - loss: 0.0182 - mae: 0.0979 - val_loss: 0.1489 - val_mae: 0.2103
Epoch 4/50
4/4 [==============================] - 2s 459ms/step - loss: 0.0173 - mae: 0.0948 - val_loss: 0.1512 - val_mae: 0.2059
Epoch 5/50
4/4 [==============================] - 2s 445ms/step - loss: 0.0165 - mae: 0.0917 - val_loss: 0.1401 - val_mae: 0.2207
Epoch 6/50
4/4 [==============================] - 2s 456ms/step - loss: 0.0155 - mae: 0.0894 - val_loss: 0.1442 - val_mae: 0.2022
Epoch 7/50
4/4 [==============================] - 2s 456ms/step - loss: 0.0143 - mae: 0.0868 - val_loss: 0.1467 - val_mae: 0.1966
Epoch 8/50
4/4 [==============================] - 2s 432ms/step - loss: 0.0140 - mae: 0.08

In [8]:
# save cnn results
cnn_results.to_csv("cnn_results.csv", index=False)

In [3]:
# get the prediction data
X_pred = np.zeros((len(ukraine_2022), 765, 1076, 1))
for i in range(len(ukraine_2022)):

    year = ukraine_2022["year"][i]
    region = ukraine_2022["region"][i]

    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"

    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]
    
    X_pred[i, :, :, 0] = allangle_snow_free

X_pred = X_pred / maximum_x

In [5]:
ukraine = pd.read_csv("data/tabular_data_ukraine.csv")
ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]
ukraine = ukraine[ukraine["year"] < 2023]
ukraine_sum = ukraine[['year', 'region', 'allangle_snow_free_hq_sum', 'allangle_snow_free_hq_mean'] + log_bin_columns]

set_seed(123)
model = define_cnn(n_features = 4, n_conv = 2, n_dense = 4)
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)

ukraine_stage_2, ukraine_pred = extract_features_2(model, ukraine, 2022, X, X_pred, 4)
selected_columns =  ["feature_" + str(i) for i in range(1, 5)] + log_bin_columns + ['allangle_snow_free_hq_sum', 'allangle_snow_free_hq_mean']

full_data = pd.concat([ukraine_stage_2, ukraine_pred])
full_data.sort_values(by=['region', 'year'], inplace=True)
full_data = pd.merge(full_data, ukraine_sum, on=['year', 'region'])
full_data_diff = full_data.groupby('region').diff()
full_data_diff['region'] = full_data['region']
full_data_diff['year'] = full_data['year']
full_data_diff.reset_index(drop=True, inplace=True)
full_data_diff = full_data_diff[full_data_diff["year"] != 2012]

train_data_diff = full_data_diff[full_data_diff["year"] != 2022]
pred_data_diff = full_data_diff[full_data_diff["year"] == 2022]
pred_data_diff.reset_index(drop=True, inplace=True)
train_data_diff.reset_index(drop=True, inplace=True)

train_data = full_data[full_data["year"] != 2022]
pred_data = full_data[full_data["year"] == 2022]
pred_data.reset_index(drop=True, inplace=True)
train_data.reset_index(drop=True, inplace=True)

# _, y_pred, best_params = predict_with_model(train_data_diff, pred_data_diff, selected_columns, "random_forest", param_grid_rf, log_transform = False, scale = False)

train_data_nn = train_data_diff.copy()
pred_data_nn = pred_data_diff.copy()

train_data_nn = pd.get_dummies(train_data_nn, columns=["region"])
pred_data_nn = pd.get_dummies(pred_data_nn, columns=["region"])

X_train_nn = train_data_nn.drop(columns=["year", "real_gdp"])
X_train_nn = X_train_nn.drop(columns=log_bin_columns)
X_train_nn = X_train_nn.drop(columns=["allangle_snow_free_hq_mean"])
y_train_nn = train_data_nn["real_gdp"]

X_test_nn = pred_data_nn.drop(columns=["year", "real_gdp"])
X_test_nn = X_test_nn.drop(columns=log_bin_columns)
X_test_nn = X_test_nn.drop(columns=["allangle_snow_free_hq_mean"])
y_test_nn = pred_data_nn["real_gdp"]

X_train_nn = np.array(X_train_nn, dtype=np.float32)
X_test_nn = np.array(X_test_nn, dtype=np.float32)
y_train_nn = np.array(y_train_nn, dtype=np.float32)
y_test_nn = np.array(y_test_nn, dtype=np.float32)


# scale the data
scaler = StandardScaler()
X_train_nn = scaler.fit_transform(X_train_nn)
X_test_nn = scaler.transform(X_test_nn)

# fit the model on the X_train and y_train
model = Sequential()
# model.add(Dense(256, activation="relu", input_dim=X_train_nn.shape[1]))  # First layer
# model.add(Dense(128, activation="relu"))  # First layer
# model.add(Dense(64, activation="relu"))  # First layer
model.add(Dense(32, activation="relu", input_dim=X_train_nn.shape[1]))  # Fourth layer
model.add(Dense(16, activation="relu"))  # New additional layer
model.add(Dense(8, activation="relu"))  # New additional layer
model.add(Dense(1)) 

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

model.fit(X_train_nn, y_train_nn, epochs=500, batch_size=64, validation_split=0.2)

# # get the predictions
# boolean_columns = [col for col in X_test_nn.columns if X_test_nn[col].dtype == bool]
# for col in boolean_columns:
#     X_test_nn[col] = X_test_nn[col].astype(int)

y_pred = model.predict(X_test_nn).flatten()

Epoch 1/50
4/4 [==============================] - 4s 645ms/step - loss: 0.0216 - mae: 0.1148 - val_loss: 0.1233 - val_mae: 0.2200
Epoch 2/50
4/4 [==============================] - 2s 438ms/step - loss: 0.0130 - mae: 0.0790 - val_loss: 0.1243 - val_mae: 0.2205
Epoch 3/50
4/4 [==============================] - 2s 437ms/step - loss: 0.0120 - mae: 0.0867 - val_loss: 0.1253 - val_mae: 0.2091
Epoch 4/50
4/4 [==============================] - 2s 430ms/step - loss: 0.0086 - mae: 0.0705 - val_loss: 0.1163 - val_mae: 0.2424
Epoch 5/50
4/4 [==============================] - 2s 429ms/step - loss: 0.0108 - mae: 0.0855 - val_loss: 0.1409 - val_mae: 0.1825
Epoch 6/50
4/4 [==============================] - 2s 422ms/step - loss: 0.0111 - mae: 0.0675 - val_loss: 0.1290 - val_mae: 0.1918
Epoch 7/50
4/4 [==============================] - 2s 433ms/step - loss: 0.0076 - mae: 0.0678 - val_loss: 0.1173 - val_mae: 0.2148
Epoch 8/50
4/4 [==============================] - 2s 417ms/step - loss: 0.0066 - mae: 0.06

In [28]:
print(sum(y_pred)/ukraine_2021["real_gdp"].sum())

pred_data_diff["gdp_prediction"] = y_pred
ukraine_2022_pred = pd.merge(ukraine_2022, pred_data_diff[["region", "year", "gdp_prediction"]], on=["region", "year"])
ukraine_2022_pred = ukraine_2022_pred[["region", "year", "gdp_prediction"]]
ukraine_2022_pred = pd.merge(ukraine_2022_pred, ukraine_2021[["region", "year", "real_gdp"]], on=["region"])
ukraine_2022_pred["gdp_prediction"] = ukraine_2022_pred["gdp_prediction"] + ukraine_2022_pred["real_gdp"]
ukraine_2022_pred = ukraine_2022_pred[["region", "gdp_prediction"]]

ukraine_2022_pred.to_csv("gdp_predictions_ukraine.csv", index=False)

-0.3072529469557337
